In [1]:
!pip install numpy==1.16.1

In [2]:
import numpy
from numpy import array
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.models import load_model
import re
import numpy as np
from nltk.tokenize import word_tokenize
import nltk

# fix random seed for reproducibility
numpy.random.seed(7)


Using TensorFlow backend.


In [3]:
top_words = 5000 #vocabulary_size = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)


17465344/17464789 [==============================] - 3s 0us/step


In [4]:
print('---review---')
print(X_train[6])
print('---label---')
print(y_train[6])

---review---
[1, 2, 365, 1234, 5, 1156, 354, 11, 14, 2, 2, 7, 1016, 2, 2, 356, 44, 4, 1349, 500, 746, 5, 200, 4, 4132, 11, 2, 2, 1117, 1831, 2, 5, 4831, 26, 6, 2, 4183, 17, 369, 37, 215, 1345, 143, 2, 5, 1838, 8, 1974, 15, 36, 119, 257, 85, 52, 486, 9, 6, 2, 2, 63, 271, 6, 196, 96, 949, 4121, 4, 2, 7, 4, 2212, 2436, 819, 63, 47, 77, 2, 180, 6, 227, 11, 94, 2494, 2, 13, 423, 4, 168, 7, 4, 22, 5, 89, 665, 71, 270, 56, 5, 13, 197, 12, 161, 2, 99, 76, 23, 2, 7, 419, 665, 40, 91, 85, 108, 7, 4, 2084, 5, 4773, 81, 55, 52, 1901]
---label---
1


In [5]:
'''We can use the dictionary returned by imdb.get_word_index() to map the review back to the original words.'''
word2id = imdb.get_word_index()
id2word = {i: word for word, i in word2id.items()}
print('---review with words---')
print([id2word.get(i, ' ') for i in X_train[6]])
print('---label---')
print(y_train[6])



1646592/1641221 [==============================] - 1s 1us/step
---review with words---
['the', 'and', 'full', 'involving', 'to', 'impressive', 'boring', 'this', 'as', 'and', 'and', 'br', 'villain', 'and', 'and', 'need', 'has', 'of', 'costumes', 'b', 'message', 'to', 'may', 'of', 'props', 'this', 'and', 'and', 'concept', 'issue', 'and', 'to', "god's", 'he', 'is', 'and', 'unfolds', 'movie', 'women', 'like', "isn't", 'surely', "i'm", 'and', 'to', 'toward', 'in', "here's", 'for', 'from', 'did', 'having', 'because', 'very', 'quality', 'it', 'is', 'and', 'and', 'really', 'book', 'is', 'both', 'too', 'worked', 'carl', 'of', 'and', 'br', 'of', 'reviewer', 'closer', 'figure', 'really', 'there', 'will', 'and', 'things', 'is', 'far', 'this', 'make', 'mistakes', 'and', 'was', "couldn't", 'of', 'few', 'br', 'of', 'you', 'to', "don't", 'female', 'than', 'place', 'she', 'to', 'was', 'between', 'that', 'nothing', 'and', 'movies', 'get', 'are', 'and', 'br', 'yes', 'female', 'just', 'its', 'because', 'm

In [6]:
print('Maximum review length: {}'.format(
len(max((X_train + X_test), key=len))))


Maximum review length: 2697


In [7]:

print('Minimum review length: {}'.format(
len(min((X_train + X_test), key=len))))

Minimum review length: 70


In [0]:
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

In [9]:
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

'''We first need to compile our model by specifying the loss function and optimizer we want to use while training, as well as any evaluation metrics 
we’d like to measure. Specify the appropriate parameters, including at least one metric ‘accuracy’.'''
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=30, batch_size=64)


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
dropout_1 (Dropout)          (None, 500, 32)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 25000 samples, validate on 25000 samples
Epoch 1/30
25000/25000 [==============================] - 522s 21ms/step - loss: 0.4521 - accuracy: 0.7832 - val_loss: 0.3671 - val_accuracy: 0.8465
Epoch 2/30
25000/25000 [==============================] - 519s 21ms/step - loss: 0.3328 - accuracy: 0.8599 - val_loss: 0.3238 - val_accuracy: 0.8678
Epoch 3/30
25000/25000 [==============================] - 514s 21ms/step - loss: 0.2675 - accuracy: 0.8949 - val_loss: 0.3365 - val_accuracy: 0.8582
Epoch 4/30
25000/25000 [==============================] - 513s 21ms/step - loss: 0.2325 - accuracy: 0.9087 - val_loss: 0.3111 - val_accuracy: 0.8724
Epoch 5/30
25000/25000 [==============================] - 511s 20ms/step - loss: 0.2010 - accuracy: 0.9239 - val_loss: 0.3120 - val_accuracy: 0.8738
Epoch 6/30
25000/25000 [==============================] - 511s 20ms/step - loss: 0.1923 - accuracy: 0.9265 - val_loss: 0.3556 - val_accuracy: 0.8671
Epoch 7/30
25000/25000 [==============================] 

In [10]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 85.92%


In [11]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 144433 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.21-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.21-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.21-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [0]:
from google.colab import auth
auth.authenticate_user()

In [13]:
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
#Mount your Google Drive:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [15]:
import os
os.mkdir("/content/drive/Sentiment_Analysis")
os.chdir("/content/drive/")
!ls

 1060060.pdf
 1ds17cs717.pdf
'2nd internals april 2020'
'5thsem project'
'6th sem'
 717_ONE.atmx
 8014258.pdf
'BTech Invitation Letter.pdf'
 cakephp
 Certificates
 Civil
'Clg results'
'Colab Notebooks'
 Cuspera
'cuspera testing project'
'data science'
 Design
 Doc1.desktop
 Doc1.docx
'documents of dsce'
 _DSC6266.JPG
 DSCE
'Ethnic 2k19'
'Ethnic day 2k18'
'Event Registration.desktop'
'Flat 993'
'GIRLSCRIPT BLR'
'Google Photos'
'HC Verma Solutions: Chapter 3 - Rest and Motion - Kinematics.pdf'
'How to get started with Drive'
 internship
'JEE(Advanced)_RegistrationDetailsPage.pdf'
 JSON
'mp lab 2019'
'MP Record doc.desktop'
'my doc'
 Placements
'Placements 2k21'
'resume-12sept (63b221e9).pdf'
'resume-12sept - Gaurang Mittal.pdf'
 resume-12sept.pdf
 Screenshot_2019-04-22-18-19-24-828_net.one97.paytm.png
 Screenshot_2019-05-08-20-17-24-477_net.one97.paytm.png
 Sentiment_Analysis
'sentiment anaylsis'
'Training Feedback - AAU.docx'
'Training Feedback - AAU - Gaurang Mittal.docx'
 UG-ML-2020
'

In [0]:
import sys
sys.path.append('/content/drive/Sentiment_Analysis')


In [17]:
model.save('/content/drive/Sentiment_Analysis/sentiment_analysis_model_new.h5')
print("Saved model to disk")


Saved model to disk


In [18]:
os.chdir("/content/drive/Sentiment_Analysis")
!ls

sentiment_analysis_model_new.h5


In [19]:
model = load_model('/content/drive/Sentiment_Analysis/sentiment_analysis_model_new.h5')
print("Model Loaded")


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Model Loaded
